In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
df=pd.read_csv('../input/credit-card-customers/BankChurners.csv')

In [ ]:
df.columns

In [ ]:
df=df.drop(columns=['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'])
df.describe

In [ ]:
df=df.replace('Unknown',np.nan)


In [ ]:
df.isnull().any()

In [ ]:
y=df[['Attrition_Flag']]
x=df[df.columns[2:]]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
ax=sns.countplot(x='Attrition_Flag',data=y)
print(y.value_counts())

Imbalance in dataset;
Evaluation for prediction should include recall and precision

In [ ]:
from sklearn.model_selection import train_test_split as tts
xtr,xte,ytr,yte=tts(x,y,test_size=0.2)

In [ ]:
yte=yte[xte.notna().all(axis=1)]
xte=xte.dropna('index')

In [ ]:
#custom distribution imputation
ms=dict(xtr['Marital_Status'].value_counts())
sintodiv=round(ms['Single']/(ms['Divorced']+ms['Single']),1)
unms=np.random.choice(['Single','Divorced'],xtr.loc[:,'Marital_Status'].isna().sum(),p=[sintodiv,1-sintodiv])
unc=0
for index,row in xtr.iterrows():
    if pd.isnull(row['Marital_Status']):
        if row['Dependent_count']==0:
            xtr.loc[index,'Marital_Status']=unms[unc]
            unc+=1
        else:
            xtr.loc[index,'Marital_Status']='Married'


In [ ]:
row['Gender']

In [ ]:
#Max imputation
xtr.loc[:,'Income_Category'].fillna('Less than $40K',inplace=True)
xtr.loc[:,'Education_Level'].fillna('Graduate',inplace=True)

In [ ]:
xtr.dtypes

In [ ]:
#encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['Less than $40K','$40K - $60K','$60K - $80K','$80K - $120K','$120K +'])
xtr.loc[:,'Income_Category']=le.transform(xtr.loc[:,'Income_Category'])
xte.loc[:,'Income_Category']=le.transform(xte.loc[:,'Income_Category'])
le.fit(['Blue','Silver','Gold','Platinum'])
xtr.loc[:,'Card_Category']=le.transform(xtr.loc[:,'Card_Category'])
xte.loc[:,'Card_Category']=le.transform(xte.loc[:,'Card_Category'])
le.fit(['Uneducated','High School','College','Graduate','Post-Graduate','Doctorate'])
xtr.loc[:,'Education_Level']=le.transform(xtr.loc[:,'Education_Level'])
xte.loc[:,'Education_Level']=le.transform(xte.loc[:,'Education_Level'])
xtr.loc[:,'Marital_Status']=le.fit_transform(xtr.loc[:,'Marital_Status'])
xte.loc[:,'Marital_Status']=le.fit_transform(xte.loc[:,'Marital_Status'])
xte.loc[:,'Gender']=le.fit_transform(xte.loc[:,'Gender'])
xtr.loc[:,'Gender']=le.fit_transform(xtr.loc[:,'Gender'])

In [ ]:
print(max(df['Customer_Age']),min(df['Customer_Age']))
hist=df['Customer_Age'].hist(bins=8)

In [ ]:
def age(cage):
    if cage<=40:
        return 0
    elif cage>40 and cage<=50:
        return 1
    elif cage>50 and cage<=60:
        return 2
    else:
        return 3

In [ ]:
for ind,row in xtr.iterrows():
    xtr.loc[ind,'Age']=age(row['Customer_Age'])
for ind,row in xte.iterrows():
    xte.loc[ind,'Age']=age(row['Customer_Age'])
xtr=xtr.drop('Customer_Age',axis=1)
xte=xte.drop('Customer_Age',axis=1)

In [ ]:
xtr_st=(xtr-xtr.mean())/xtr.std()
xtrn=pd.concat([ytr,xtr_st.iloc[:,0:10]],axis=1)
xtrn=pd.melt(xtrn,id_vars='Attrition_Flag',var_name='features',value_name='value')
plt.figure(figsize=(12,12))
#tic=time.time()
sns.violinplot(x='features',y='value',hue='Attrition_Flag',data=xtrn,split=True,inner='quart')
plt.xticks(rotation=90)

In [ ]:
xtrn=pd.concat([ytr,xtr_st.iloc[:,10:]],axis=1)
xtrn=pd.melt(xtrn,id_vars='Attrition_Flag',var_name='features',value_name='value')
plt.figure(figsize=(12,12))
#tic=time.time()
sns.violinplot(x='features',y='value',hue='Attrition_Flag',data=xtrn,split=True,inner='quart')
plt.xticks(rotation=90)

In [ ]:
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(xtr.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
#feature selection
features=['Total_Revolving_Bal','Total_Amt_Chng_Q4_Q1','Total_Trans_Amt','Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio']
xtefs=xte[features]
xtrfs=xtr[features]

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(['Attrited Customer','Existing Customer'])
ytr=le.transform(ytr['Attrition_Flag'].values.reshape((len(ytr),)))
yte=le.transform(yte['Attrition_Flag'].values.reshape((len(yte),)))

In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score,f1_score

model = RandomForestClassifier(random_state=1)
model.fit(xtrfs, ytr)
preds = model.predict(xtefs)

for i in range(len(preds)):
    if preds[i]<0.5:
        preds[i]=0
    else:
        preds[i]=1

print('MAE :',mean_absolute_error(yte, preds))
print('Accuracy :',accuracy_score(yte,preds))

In [ ]:
from sklearn.metrics import confusion_matrix as cm
cm1=cm(yte,preds)
sns.heatmap(cm1,annot=True,fmt='d')
print('F1 score :',f1_score(yte,preds))